In [1]:
cd "C:/Users/kangyuwang/OneDrive/portfolio/python_geo/Lesson 5"

C:\Users\kangyuwang\OneDrive\portfolio\python_geo\Lesson 5


In [19]:
#Import packages
from bokeh.palettes import YlOrRd6 as palette
from bokeh.plotting import figure, save
from bokeh.models import ColumnDataSource, HoverTool, LogColorMapper, GeoJSONDataSource
from bokeh.palettes import RdYlGn10 as palette
import geopandas as gpd
import pysal as ps
import numpy as np
import pandas as pd
import mapclassify

In [25]:
#Read data
data=gpd.read_file("dataE5/TravelTimes_to_5975375_RailwayStation.shp")
roads=gpd.read_file("dataE5/roads.shp")
metro=gpd.read_file("dataE5/metro.shp")

In [26]:
#Adjust CRS
data=data.to_crs(epsg=3067)
roads=roads.to_crs(epsg=3067)
metro=metro.to_crs(epsg=3067)

In [27]:
#Create x and y coordinates
roads.loc[39, "geometry"]=roads.loc[39, "geometry"][0]
roads.loc[158, "geometry"]=roads.loc[158, "geometry"][0]
data.loc[:, "x"]=data.loc[:, "geometry"].apply(lambda f: list(f.exterior.xy[0]))
data.loc[:, "y"]=data.loc[:, "geometry"].apply(lambda f: list(f.exterior.xy[1]))
metro.loc[:, "x"]=metro.loc[:, "geometry"].apply(lambda f: list(f.xy[0]))
metro.loc[:, "y"]=metro.loc[:, "geometry"].apply(lambda f: list(f.xy[1]))
roads.loc[:, "x"]=roads.loc[:, "geometry"].apply(lambda f: list(f.xy[0]))
roads.loc[:, "y"]=roads.loc[:, "geometry"].apply(lambda f: list(f.xy[1]))

In [28]:
#Replace No Data values
data=data.replace(-1, 999)

In [29]:
#Classify travel times
breaks=[x for x in range(5, 200, 5)]
classifier=mapclassify.User_Defined.make(bins=breaks)
pt_classify=classifier(data.loc[:, "pt_r_tt"])
car_classify=classifier(data.loc[:, "car_r_t"])
data.loc[:, "pt_r_tt_ud"] = pt_classify
data.loc[:, "car_r_t_ud"] = car_classify
data.head()

,car_m_d,car_m_t,car_r_d,car_r_t,from_id,pt_m_d,pt_m_t,pt_m_tt,pt_r_d,pt_r_t,pt_r_tt,to_id,walk_d,walk_t,geometry,x,y,pt_r_tt_ud,car_r_t_ud
0,32297,43,32260,48,5785640,32616,116,147,32616,108,139,5975375,32164,459,"POLYGON ((382000.0001358641 6697750.000038058,...","[382000.00013586413, 381750.0001359122, 381750...","[6697750.000038058, 6697750.000038066, 6698000...",27,9
1,32508,43,32471,49,5785641,32822,119,145,32822,111,133,5975375,29547,422,"POLYGON ((382250.0001358146 6697750.000038052,...","[382250.0001358146, 382000.00013586413, 382000...","[6697750.000038052, 6697750.000038058, 6698000...",26,9
2,30133,50,31872,56,5785642,32940,121,146,32940,113,133,5975375,29626,423,"POLYGON ((382500.0001357661 6697750.000038046,...","[382500.0001357661, 382250.0001358146, 382250....","[6697750.000038046, 6697750.000038052, 6698000...",26,11
3,32690,54,34429,60,5785643,33233,125,150,33233,117,144,5975375,29919,427,"POLYGON ((382750.0001357181 6697750.000038039,...","[382750.0001357181, 382500.0001357661, 382500....","[6697750.000038039, 6697750.000038046, 6698000...",28,11
4,31872,42,31834,48,5787544,32127,109,126,32127,101,121,5975375,31674,452,"POLYGON ((381250.0001360176 6697500.00003812, ...","[381250.00013601757, 381000.00013606605, 38100...","[6697500.00003812, 6697500.000038126, 6697750....",24,9


In [30]:
#Give names to time ranges
names=[f"{n}-{n+5}" for n in range(0, 60, 5)]
names.append("60 <")

In [31]:
names

['0-5',
 '5-10',
 '10-15',
 '15-20',
 '20-25',
 '25-30',
 '30-35',
 '35-40',
 '40-45',
 '45-50',
 '50-55',
 '55-60',
 '60 <']

In [32]:
names_df=pd.DataFrame(data=names)
names_df.columns=["name"]
names_df.loc[:, "index"]=names_df.index

In [33]:
data=data.merge(right=names_df, how="left", left_on="pt_r_tt_ud", right_on="index")
data.loc[:, "pt_name"]=data.loc[:, "name"]
data=data.drop(["index", "name"], axis=1)

data=data.merge(right=names_df, how="left", left_on="car_r_t_ud", right_on="index")
data.loc[:, "car_name"]=data.loc[:, "name"]
data=data.drop(["index", "name"], axis=1)

In [34]:
data.loc[data.loc[:, "car_name"].isnull(), "car_name"]="60 <"
data.loc[data.loc[:, "pt_name"].isnull(), "pt_name"]="60 <"

In [51]:
#Make maps
df=data.loc[:, ['x', 'y', 'pt_r_tt_ud', 'pt_r_tt', 'car_r_t', 'from_id', 'pt_name']]
df_source=ColumnDataSource(data=df)

rdf=roads.loc[:, ["x", "y"]]
rdf_source=ColumnDataSource(data=rdf)

mdf=metro.loc[:, ["x", "y"]]
mdf_source=ColumnDataSource(data=mdf)

TOOLS="pan, wheel_zoom, box_zoom, reset, save"

pallete.reverse()
color_mapper=LogColorMapper(palette=palette)

p=figure(title="Travel times to Helsinki city center by public transportation", tools=TOOLS, plot_width=650, plot_height=500, active_scroll="wheel_zoom")
p.grid.grid_line_color=None
grid=p.patches("x", "y", source=df_source, name="grid", fill_color={"field":"pt_r_tt_ud", "transform":color_mapper}, fill_alpha=1.0, line_color="black", line_width=0.03, legend="pt_name")
r=p.multi_line("x", "y", source=rdf_source, color="grey")
m=p.multi_line("x", "y", source=mdf_source, color="red")

p.legend.location="top_right"
p.legend.orientation="vertical"

circle=p.circle(x=385752.214, y=6672143.803, name="point", size=6, color="yellow")

phover=HoverTool(renderers=[circle])
phover.tooltips=[("Destination", "Railway Station")]

ghover=HoverTool(renderers=[grid])
ghover.tooltips=[("YKR-ID", "@from_id"), ("PT time", "@pt_r_tt"), ("Car time", "@car_r_t")]

p.add_tools(ghover)
p.add_tools(phover)

save(p, "accessibility_map_Helsinki.html")

C:\ProgramData\Anaconda3\lib\site-packages\bokeh\io\saving.py:126: UserWarning: save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN
  warn("save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN")
C:\ProgramData\Anaconda3\lib\site-packages\bokeh\io\saving.py:139: UserWarning: save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'
  warn("save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'")


'C:\\Users\\kangyuwang\\OneDrive\\portfolio\\python_geo\\Lesson 5\\accessibility_map_Helsinki.html'